In [5]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [6]:
data = pd.read_csv("t20_data.csv.zip", compression="zip")

In [7]:
data.sample(3)

,match_id,venue,toss_winner,toss_decision,batting_team,bowling_team,innings,runs_so_far,wickets_so_far,balls_faced,run_rate,target_runs,required_run_rate,winner
56046,1394797,Vanuatu Cricket Ground (Oval 2),Indonesia,field,Indonesia,Vanuatu,2,7,1,16,2.333333,86.0,4.557692,Vanuatu
658408,1451318,"Yeonhui Cricket Ground, Incheon",South Korea,bat,South Korea,China,1,27,7,68,2.219178,NaN,NaN,China
535893,1418182,"Terdthai Cricket Ground, Bangkok",Singapore,bat,Maldives,Singapore,2,5,0,3,5.000000,200.0,10.000000,Singapore


In [8]:
data["batting_team_wins"] = (data["winner"] == data["batting_team"]).astype(int)
data.sample(3)

,match_id,venue,toss_winner,toss_decision,batting_team,bowling_team,innings,runs_so_far,wickets_so_far,balls_faced,run_rate,target_runs,required_run_rate,winner,batting_team_wins
770571,1233955,Bay Oval,West Indies,field,West Indies,New Zealand,2,51,2,42,7.116279,239.0,14.461538,New Zealand,0
434064,1284489,"West End Park International Cricket Stadium, Doha",Maldives,bat,Maldives,Saudi Arabia,1,3,0,6,3.000000,NaN,NaN,Saudi Arabia,0
126290,1283042,Gahanga International Cricket Stadium. Rwanda,Botswana,bat,Botswana,Sierra Leone,1,44,1,36,7.135135,NaN,NaN,Botswana,1


In [9]:
# Fill missing numeric with -1
numeric_cols = data.select_dtypes(include=[np.number]).columns
data[numeric_cols] = data[numeric_cols].fillna(-1)

In [10]:
from sklearn.preprocessing import LabelEncoder

In [11]:
# Categorical encoding
categorical_cols = ["venue", "batting_team", "bowling_team", "toss_winner", "toss_decision", "winner"]
label_encoders = {}

for col in categorical_cols:
    le = LabelEncoder()
    data[col] = le.fit_transform(data[col].astype(str))
    label_encoders[col] = le

In [12]:
data.sample(3)

,match_id,venue,toss_winner,toss_decision,batting_team,bowling_team,innings,runs_so_far,wickets_so_far,balls_faced,run_rate,target_runs,required_run_rate,winner,batting_team_wins
577562,1453934,135,50,0,51,32,1,140,4,95,8.571429,-1.0,-1.000000,48,1
219086,1407100,390,67,0,100,68,2,28,1,33,4.666667,100.0,4.965517,94,1
995903,895817,4,1,1,1,41,2,90,3,58,9.000000,189.0,9.580645,40,0


In [13]:
df_inn1 = data[data["innings"] == 1].copy()
df_inn2 = data[data["innings"] == 2].copy()

# Features for innings 1 → drop target-related
features_inn1 = [col for col in df_inn1.columns if col not in [
    "match_id", "winner", "batting_team_wins", 
    "target_runs", "runs_required", "required_rr"
]]

# Features for innings 2 → keep all
features_inn2 = [col for col in df_inn2.columns if col not in [
    "match_id", "winner", "batting_team_wins"
]]

In [14]:
from sklearn.model_selection import GroupShuffleSplit

# Split so same match doesn't leak between train/test
gss1 = GroupShuffleSplit(n_splits=1, test_size=0.2, random_state=42)
train_idx1, test_idx1 = next(gss1.split(df_inn1[features_inn1], df_inn1["batting_team_wins"], groups=df_inn1["match_id"]))

gss2 = GroupShuffleSplit(n_splits=1, test_size=0.2, random_state=42)
train_idx2, test_idx2 = next(gss2.split(df_inn2[features_inn2], df_inn2["batting_team_wins"], groups=df_inn2["match_id"]))

X1_train, X1_test = df_inn1[features_inn1].iloc[train_idx1], df_inn1[features_inn1].iloc[test_idx1]
y1_train, y1_test = df_inn1["batting_team_wins"].iloc[train_idx1], df_inn1["batting_team_wins"].iloc[test_idx1]

X2_train, X2_test = df_inn2[features_inn2].iloc[train_idx2], df_inn2[features_inn2].iloc[test_idx2]
y2_train, y2_test = df_inn2["batting_team_wins"].iloc[train_idx2], df_inn2["batting_team_wins"].iloc[test_idx2]

print("Train size:", X1_train.shape, "Test size:", X1_test.shape)
print("Train size:", X2_train.shape, "Test size:", X2_test.shape)

Train size: (439135, 11) Test size: (109610, 11)
Train size: (374039, 12) Test size: (94765, 12)


In [33]:
from lightgbm import LGBMClassifier

# First innings model
model_inn1 = LGBMClassifier(
    objective="binary",
    learning_rate=0.002,
    num_leaves=31,
    n_estimators=1200,
    random_state=42
)

model_inn1.fit(
    X1_train, y1_train,
    eval_set=[(X1_test, y1_test)],
    eval_metric="auc",
)


[LightGBM] [Info] Number of positive: 218554, number of negative: 220581
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.004374 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 1195
[LightGBM] [Info] Number of data points in the train set: 439135, number of used features: 9
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.497692 -> initscore=-0.009232
[LightGBM] [Info] Start training from score -0.009232


LGBMClassifier(learning_rate=0.002, n_estimators=1200, objective='binary',
               random_state=42)

In [37]:
# Second innings model
model_inn2 = LGBMClassifier(
    objective="binary",
    learning_rate=0.002,
    num_leaves=31,
    n_estimators=1000,
    random_state=42
)

model_inn2.fit(
    X2_train, y2_train,
    eval_set=[(X2_test, y2_test)],
    eval_metric="auc",
)

[LightGBM] [Info] Number of positive: 169417, number of negative: 204622
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.004611 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 1687
[LightGBM] [Info] Number of data points in the train set: 374039, number of used features: 11
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.452939 -> initscore=-0.188801
[LightGBM] [Info] Start training from score -0.188801


LGBMClassifier(learning_rate=0.002, n_estimators=1000, objective='binary',
               random_state=42)

In [38]:
from sklearn.metrics import roc_auc_score, accuracy_score

# First innings
y1_pred_proba = model_inn1.predict_proba(X1_test)[:, 1]
y1_pred = (y1_pred_proba >= 0.5).astype(int)
print("First Innings AUC:", roc_auc_score(y1_test, y1_pred_proba))
print("First Innings Accuracy:", accuracy_score(y1_test, y1_pred))

# Second innings
y2_pred_proba = model_inn2.predict_proba(X2_test)[:, 1]
y2_pred = (y2_pred_proba >= 0.5).astype(int)
print("Second Innings AUC:", roc_auc_score(y2_test, y2_pred_proba))
print("Second Innings Accuracy:", accuracy_score(y2_test, y2_pred))

First Innings AUC: 0.7396501442297218
First Innings Accuracy: 0.6746191040963416
Second Innings AUC: 0.9124063503537123
Second Innings Accuracy: 0.8268981163931831


In [40]:
import pickle

model_bundle = {
    "model_inn1": model_inn1,
    "model_inn2": model_inn2,
    "label_encoders": label_encoders,
    "features_inn1": features_inn1,
    "features_inn2": features_inn2
}

with open("pipe.pkl", "wb") as f:
    pickle.dump(model_bundle, f)

print("✅ Saved two-model setup to pipe.pkl")


✅ Saved two-model setup to pipe.pkl


In [18]:
data.head()

,match_id,venue,toss_winner,toss_decision,batting_team,bowling_team,innings,runs_so_far,wickets_so_far,balls_faced,run_rate,target_runs,required_run_rate,winner,batting_team_wins
0,1442989,258,89,1,41,90,1,0,0,1,0.0,-1.0,-1.0,100,0
1,1442989,258,89,1,41,90,1,1,0,2,1.0,-1.0,-1.0,100,0
2,1442989,258,89,1,41,90,1,1,0,3,1.0,-1.0,-1.0,100,0
3,1442989,258,89,1,41,90,1,2,0,4,2.0,-1.0,-1.0,100,0
4,1442989,258,89,1,41,90,1,3,0,5,3.0,-1.0,-1.0,100,0
